# Stage 1: Setup - Step 1: Initialize

This step initializes the workflow and validates the environment.

**Purpose:**
- Verify database connectivity
- Validate cycle configuration
- Set up working directories
- Initialize logging

In [2]:
# Step tracking initialization
%run /workspace/system/helpers/03_StepTracker.ipynb
%run /workspace/system/helpers/04_Display.ipynb

✅ Configuration loaded - Base path: /workspace
✅ Database connection successful
✅ Step Tracker loaded
✅ Display utilities loaded


In [13]:
from system.helpers.workflow_context import get_workflow_context
context = get_workflow_context()
step = StepTracker(cycle_name=context.cycle_name, stage_num=context.stage_num, step_num=context.step_num, idempotent=False, auto_start=True)
if not step.can_execute:
    print("Step cannot be executed. Check the message above.")

⚠️  Step is currently running
Step cannot be executed. Check the message above.


## Step Execution

In [10]:
if step.can_execute:
    try:
        print_header("Initializing Workflow")
        
        # 1. Test database connection
        print("\n1. Testing database connection...")
        if test_connection():
            print("   ✅ Database connected")
        else:
            raise Exception("Database connection failed")
        
        # 2. Verify active cycle
        print("\n2. Verifying active cycle...")
        active = get_active_cycle()
        if active:
            print(f"   ✅ Active cycle: {active['cycle_name']}")
        else:
            raise Exception("No active cycle found")
        
        # 3. Set up working directories
        print("\n3. Setting up directories...")
        from pathlib import Path
        
        cycle_dir = Path(f"/workspace/workflows/Active_{active['cycle_name']}")
        files_dir = cycle_dir / "files"
        logs_dir = cycle_dir / "logs"
        
        files_dir.mkdir(exist_ok=True)
        logs_dir.mkdir(exist_ok=True)
        print(f"   ✅ Files directory: {files_dir}")
        print(f"   ✅ Logs directory: {logs_dir}")
        
        # 4. Initialize logging
        print("\n4. Initializing logging...")
        import logging
        from datetime import datetime
        
        log_file = logs_dir / f"initialization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
        logging.basicConfig(
            filename=str(log_file),
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        logging.info("Workflow initialized")
        print(f"   ✅ Log file: {log_file}")
        
        # Store output data
        output = {
            'cycle_name': active['cycle_name'],
            'cycle_id': active['id'],
            'files_dir': str(files_dir),
            'logs_dir': str(logs_dir),
            'log_file': str(log_file)
        }
        
        # Mark step as completed
        step.complete_run(output)
        
        print("\n" + "=" * 60)
        print("✅ INITIALIZATION COMPLETE")
        print("=" * 60)
        
    except Exception as e:
        # Mark step as failed
        step.fail_run(str(e))
        print(f"\n❌ Step failed: {e}")
        raise

 Initializing Workflow

1. Testing database connection...
   ✅ Database connected

2. Verifying active cycle...
   ✅ Active cycle: Q1_2024_Analysis

3. Setting up directories...
   ✅ Files directory: /workspace/workflows/Active_Q1_2024_Analysis/files
   ✅ Logs directory: /workspace/workflows/Active_Q1_2024_Analysis/logs

4. Initializing logging...
   ✅ Log file: /workspace/workflows/Active_Q1_2024_Analysis/logs/initialization_20251004_024857.log
✅ Step 1.2 completed

✅ INITIALIZATION COMPLETE


## Verification

In [11]:
# Verify step completion
print_header("Step Status")

history = get_step_history(step.cycle_name, step.stage_num, step.step_num)
if not history.empty:
    display_dataframe(history.head(5), title="Step Run History")
else:
    print("No run history available")

 Step Status
------------------------------------------------------------
 Step Run History
------------------------------------------------------------
+-------------+--------------+------------+-------------+--------------+-----------+---------------------+---------------------+---------------+--------------------+
|   stage_num | stage_name   |   step_num | step_name   |   run_number | status    | started_ts          | completed_ts        | started_by    | duration_seconds   |
+=============+==============+============+=============+==============+===========+=====================+=====================+===============+====================+
|           1 | Setup        |          2 | Validate    |            1 | completed | 2025-10-04 02:48:47 | 2025-10-04 02:48:57 | notebook_user | 9s                 |
+-------------+--------------+------------+-------------+--------------+-----------+---------------------+---------------------+---------------+--------------------+
